### signal simulation

Here we demonstrate simulation of the GPS L1 CA signal. We assume that the carrier has been downconverted to around 1.25MHz, and then sampled at a rate of 5MHz. During the downconversion process, in-phase and quadrature phase signals were generated. We will simulate as if we only use one of these.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from gnss import codes
from gnss import signals

We will want to be able to create and store a signal struct that generically defines the parameters of the signal (this is esspecially important when designing a generic receiver architecture).

In [1]:
%%writefile ../../gnss/signals/__init__.py


from gnss.codes import gps_l1, gps_l2, gps_l5

GPSL1_CARRIER_FREQUENCY = 1.57542e9
GPSL2_CARRIER_FREQUENCY = 1.2276e9
GPSL5_CARRIER_FREQUENCY = 1.17645e9

class Signal:
    """
    Defines attributes of a GNSS signal, which is comprised of one code modulated on one carrier frequency.
    
    
    -8 is unkown GLONASS frequency number.
    """
    
    def __init__(self, svid, f_carrier, code, f_nav=None, code_nav=None, signal_type='', freq_no=-8):
        self.f_carrier = f_carrier
        self.code = code
        self.f_nav = f_nav
        self.code_nav = code_nav
        self.signal_type = signal_type
    
    def GPSL1CA(svid):
        code = gps_l1.gps_l1ca(svid)
        return Signal(svid, GPSL1_CARRIER_FREQUENCY, code, 50., signal_type='GPSL1CA')
    
    def GPSL2(svid):
        code = gps_l2.gps_l2c(svid)
        return Signal(svid, GPSL2_CARRIER_FREQUENCY, code, signal_type='GPSL2C')
      
    def GPSL5_I(svid):
        code_i = gps_l5.gps_l5i(svid)
        return Signal(svid, GPSL5_CARRIER_FREQUENCY, code_i, signal_type='GPSL5I')
    
    def GPSL5_Q(svid):
        # TODO describe nav data
        code_q = gps_l5.gps_l5q(svid)
        return Signal(svid, GPSL5_CARRIER_FREQUENCY, code_q, signal_type='GPSL5Q')
    

Overwriting ../../gnss/signals/__init__.py


In [3]:
svid = 1
signal = signals.Signal.GPSL1CA(svid)

In [9]:
svid = 1         # SV id
duration = 1e-3  # signal duration
f_samp = 5e6     # sample rate
f_center = 1.25e6# downconverted carrier frequency at time of sampling
f_dopp = 1000.   # doppler frequency added to carrier frequency
phi0 = 0.        # initial carrier phase offset at time of sampling, in radians

# initial code phase offset (in chips)
n0 = 512.  # initial code sample phase (used for easy verification in correlation results)
f_chip = signal.code.rate * (1. + f_dopp / signal.f_carrier)
c0 = (n0 / f_samp) * f_chip  # chip phase equals time offset / doppler adjusted chip period

# carrier-to-noise ratio
cn0 = 49.

In [10]:
# number of samples
n = int(duration * f_samp)
print('number of samples: {0}'.format(n))
# time array
t = np.arange(n) / f_samp

number of samples: 5000


In [12]:
code_samples = 1 - 2 * signal.code.sequence[(np.floor(c0 + t * f_chip) % len(signal.code.sequence)).astype(int)]
signal_samples = code_samples * np.exp(2j * np.pi * (f_center + f_dopp) * t + 1j * phi0)
baseband_samples = signal_samples * np.exp(-2j * np.pi * f_center * t)

In [29]:
start, end = 0, 500
plt.subplot(121)
plt.plot(np.real(signal_samples[start:end]))
plt.plot(code_samples[start:end])
plt.title('signal')

plt.subplot(122)
plt.plot(np.real(baseband_samples[start:end]))
plt.title('downconverted signal')
plt.xlabel('sample number')
plt.show()

To make this functionality easily reusable, we will make a function for simulating receiver signals.

In [18]:
%%writefile ../../gnss/signals/simulate.py


from numpy import sin, cos, floor, ones, arange

def generate_signal(signal, duration=1e-3, f_samp=5e6, f_center=1.25e6, f_dopp=0., code_phase, phi0=0., real=True):
    """
    Generates `n` signal samples, where `n` is `duration * fs`. The carrier is
    multiplied by mapped code samples, where `0` maps to `1` and `1` maps to `-1`.
    This effectively implements BPSK of the code onto the carrier. If `real` is true,
    (default) then the carrier is just a cosine, otherwise, the carrier is a complex
    expontential.
    
    Parameters
    ------
    duration: float
        signal duration in seconds
    f_samp : float
        receiver sampling rate (default 5MHz)
    f_center: float
        front-end center frequency
        (default 1.25MHz)
    fd: float
        the doppler frequency added to signal carrier frequency (default 0Hz)
    code_phase: float
        the initial code phase (in chips
    phi: float
        the initial phase offset of the carrier signal in radians (default 0rad)
    real: boolean (default True)
        if True, returns real sinusoid modulated by code, otherwise returns complex
        exponential modulated by code.
        
    """
    # we create a time vector according to user specified sampling rate and duration
    n = int(duration * f_samp)
    t = arange(n) / f_samp
    
    # samples will store the simulated signal, we use in-phase samples if real=True
    i_samples = ones((n,), dtype=float)
    if not real:
        q_samples = ones((n,), dtype=float)

    # the doppler frequency affects the chip rate by a factor of (1 + fd/fc)
    # the code indices are detemined by taking the floor of the sampling time
    # divided by the doppler frequency adjusted chip period, which is equivalent
    # to multiplying by the corrected chip rate.
    if i_code:
        indices = (floor(i_code_phase + t * (i_code.rate * (1. + fd / fc))) % len(i_code.sequence)).astype(int)
        i_samples *= 1 - 2 * i_code.sequence[indices]
    if q_code and not real:
        indices = (floor(q_code_phase + t * (q_code.rate * (1. + fd / fc))) % len(q_code.sequence)).astype(int)
        q_samples *= 1 - 2 * q_code.sequence[indices]
    
    # we multiply the samples by either a real or complex sinusoid
    if real:
        i_samples *= cos(2 * np.pi * (fi + fd) * t + phi)
        return i_samples
    else:
        i_samples *= cos(2 * np.pi * (fi + fd) * t + phi)
        q_samples *= sin(2 * np.pi * (fi + fd) * t + phi)
        return i_samples + 1j * q_samples

We have that $C/N_0 = 10 \log_{10}(\frac{P_s}{P_n}) = 10\log_{10}P_s - 10\log_{10}P_n$

The noise power $P_n = kT_k$ where $k = 1.38*10^{-23}$ is Boltzmann's constant.

We estimate $T_k = T_{ANT} + T_{RX}*(1-NF) \approx 100^\circ + 435^\circ \approx 535^\circ K$, where $T_{ANT}$ is the antenna sky noise, and $T_{RX}$ is the receiver thermal noise, which we multiply by $(1-NF)$, with $NF$ being the receiver noise figure.

With these assumptions, we find $P_n = k T_k \approx -201 dBW$.

A nominal $C/N_0$ ratio is around $49dW$.

In [19]:
def add_noise(signals, cn0s=[49.], tk=535., rx_bandwidth=2e6):
    """
    Adds noise to a signal.
    
    Parameters
    ----------
    signals: list of ndarrays of shape (N,)
        signal vectors that will be modified by an amplitude term and added together
    cn0s: list of floats
        signal-to-noise ratios (in dbHz), which typically ranges between 33-55 dbHz,
        for corresponding signal in signals list. (nominal 49)
    tk: float
        receiver system noise temperature, which is a combination of sky noise and
        thermal noise in the receiver. (default 535)
    rx_bandwidth: float
        receiver bandwidth (default 2MHz)
    
    Notes
    -----
    """
    # define Boltzmann's constant
    k = 1.38e-23
    noise_pwr = k * tk
    # calculate signal amplitude using relationship Ps = 1/2 A^2 and
    # CN0 = 10 * log(Ps/Pn) with Ps in Watts and Pn in Watts/Hz
    amplitudes = [np.sqrt(2 * noise_pwr) * 10 ** (cn0 / 20.) for cn0 in cn0s]
    samples = np.zeros(signals[0].shape)
    for signal, a in zip(signals, amplitudes):
        samples += a * signal
    # noise amplitude depends on receiver bandwidth
    noise_var = noise_pwr * rx_bandwidth
    return samples + np.sqrt(noise_var) * np.random.randn(n)

And a function to quantize a signal as well.

In [20]:
def quantize(signal, bits=4, signal_level=1.):
    """
    Quantizes a signal.
    
    Parameters
    ----------
    bits: int
        bits in ADC--the number of signal quantization levels will be 2^`bits`
    signal_level: float
        the tuned incoming signal level relative to ADC max voltage
        
    Notes
    -----
    """
    levels = 2 ** (bits - 1)
    return np.floor(levels * signal / np.max(signal))

We can use our new functions like this:

In [30]:
ca_code = code_gen.gps_l1_ca(sv_id)
codes = [(ca_code, 1.023e6, 0., False)]

code_samples = generate_signal(fs=fs, duration=duration, fc=fc, fi=0., fd=0., phi=0., codes=codes)
signal = generate_signal(fs=fs, duration=duration, fc=fc, fi=fi, fd=fd, phi=phi, codes=codes, real=False)
signal = add_noise(signals=[signal], cn0s=[70.])
signal = quantize(signal)

In [31]:
downconverted_signal = signal * generate_signal(fs=fs, duration=duration, fc=fc, fi=-fi, 
                                                     fd=-fd, phi=phi, codes=[], real=False)

Plotting our results, we can see the high frequency components as well as the code modulation components of the signal.

In [32]:
plt.subplot(131)
plt.plot(code_samples[:100])
plt.title('code')

plt.subplot(132)
plt.plot(np.abs(signal[:100]))
plt.title('signal')

plt.subplot(133)
plt.plot(np.abs(downconverted_signal[:100]))
plt.title('downconverted signal')
plt.xlabel('sample number')
plt.show()

We can see that the downconverted signal still has high-frequency components that inhibit the recovery of the CA code. We can fix this by filtering the signal.

In [24]:
signal_freq = np.fft.fft(downconverted_signal)
plt.plot(np.abs(signal_freq))
plt.show()

We can recover the CA code by filtering. (WHY DOES THE GAUSSIAN LPF WORK???)

In [26]:
# lpf = np.ones(signal_freq.shape)
# lpf[1250:3750] = 0. # high frequencies in middle
lpf = 1. - np.exp(-0.5 * ((np.arange(n) - n / 2) / 1000.)**2)  # discard pdf amplitude 1./(1000. * np.sqrt(2 * np.pi))
filtered_freq = lpf * signal_freq
filtered_signal = np.fft.ifft(filtered_freq)

In [27]:
plt.subplot(231)
plt.plot(np.fft.fftshift(np.fft.fft(signal)))
plt.title('original signal spectrum')
plt.subplot(232)
plt.plot(np.fft.fftshift(signal_freq))
plt.title('downconverted signal spectrum')
plt.subplot(233)
plt.plot(np.fft.fftshift(lpf))
plt.title('filter spectrum')
plt.subplot(234)
plt.plot(downconverted_signal[:100])
plt.title('downconverted signal')
plt.subplot(235)
plt.plot(np.fft.fftshift(filtered_freq))
plt.title('filtered signal spectrum')
plt.subplot(236)
plt.plot(filtered_signal[:100])
plt.title('filtered signal')
plt.show()

/usr/lib/python2.7/dist-packages/numpy/core/numeric.py:460: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [28]:
plt.subplot(231)
plt.plot(np.real(downconverted_signal[:250]))
plt.title('real downconverted signal')
plt.subplot(232)
plt.plot(np.imag(downconverted_signal[:250]))
plt.title('imag downconverted signal')
plt.subplot(233)
plt.plot(np.absolute(downconverted_signal[:250]))
plt.title('abs downconverted signal')
plt.subplot(234)
plt.plot(np.real(filtered_signal[:250]))
plt.title('real filtered signal')
plt.subplot(235)
plt.plot(np.imag(filtered_signal[:250]))
plt.title('imag filtered signal')
plt.subplot(236)
plt.plot(np.absolute(filtered_signal[:250]))
plt.title('abs filtered signal')
plt.show()